# Overlapping schwarz decomposition for DCOPF

* Load data

In [1]:
using LightGraphs, Random, LinearAlgebra, Printf, DelimitedFiles, SparseArrays, PowerModels

In [2]:
PowerModels.silence()

[info | PowerModels]: Suppressing information and warning messages for the rest of this session.  Use the Memento package for more fine-grained control of logging.


In [5]:
function get_data(path)
    data = PowerModels.parse_file(path)
    A = PowerModels.calc_susceptance_matrix(data)

    args = Dict()
    args[:N] = size(A.matrix, 1) # number of buses
    args[:y] = A.matrix
    args[:del] = spzeros(args[:N],args[:N]) # 
    args[:g] = LightGraphs.Graph(args[:N]) # the lightgraph; assign the nodes
    for e in values(data["branch"])
        i = A.bus_to_idx[e["f_bus"]]
        j = A.bus_to_idx[e["t_bus"]]
        LightGraphs.add_edge!(args[:g],i,j) # assign the edges.
        args[:del][i,j] = e["angmin"]
        args[:del][j,i] = e["angmin"]
    end

    # voltage angle limits
    args[:val] = -ones(args[:N])*Inf # lower bound
    args[:vau] = ones(args[:N])*Inf # upper bound
    args[:ref] = A.bus_to_idx[PowerModels.reference_bus(data)["index"]] # reference buses
    args[:val][args[:ref]] = 0; args[:vau][args[:ref]] = 0 # the voltage angle of reference bus is 0.


    gens = PowerModels.bus_gen_lookup(data["gen"], data["bus"]) # bus index 중 generator bus index에 대해서만 관련 PV bus 정보 제공하는 함수. (vg, mbase, source_id, pg, model, shutdown, startup, index, cost, qg)
    # generator cost coefficients
    args[:c1] = [[1e6, -1e6] for i=1:args[:N]]
    args[:c2] = [[.0, .0] for i=1:args[:N]]
    args[:sl] = [[.0,-1e2] for i=1:args[:N]] # generator power min?
    args[:su] = [[1e2,.0] for i=1:args[:N]] # generator power max?
    args[:ng] = 2*ones(Int64,args[:N]) # number of generators? <= TODO: Need to check this.

    for i=1:args[:N] # for each bus
        bus = A.idx_to_bus[i] # return to the bus index from the index of matrix.
        for j=1:length(gens[bus]) # 해당 PV bus에 있는 발전기 개수 별 loop?
            if length(gens[bus][j]["cost"]) != 0
                # push! <= list append와 같은 개념
                push!(args[:c1][i], gens[bus][j]["cost"][1])
                push!(args[:c2][i], gens[bus][j]["cost"][2])
                push!(args[:sl][i], gens[bus][j]["pmin"])
                push!(args[:su][i], gens[bus][j]["pmax"])
                args[:ng][i] += 1
            end
        end
    end

    args[:Ng] = sum(args[:ng]) # <= TODO: Need to check this.

    # PQ bus의 Pd 저장?
    args[:sd] = zeros(args[:N])
    for v in values(data["load"])
        args[:sd][A.bus_to_idx[v["load_bus"]]] = v["pd"]
    end

    return args
end

get_data (generic function with 1 method)

In [6]:
path = "/home/kjsong/workspace_skj/Plasmo_tutorial/data/pglib_opf_case9241_pegase.m"

args = get_data(path)

Dict{Any, Any} with 14 entries:
  :su  => [[100.0, 0.0], [100.0, 0.0, 1.0], [100.0, 0.0], [100.0, 0.0], [100.0,…
  :vau => [Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf, Inf  …  Inf, Inf, Inf, …
  :c1  => [[1.0e6, -1.0e6], [1.0e6, -1.0e6, 4814.49], [1.0e6, -1.0e6], [1.0e6, …
  :c2  => [[0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.…
  :ng  => [2, 3, 2, 2, 2, 3, 2, 3, 2, 2  …  2, 2, 2, 3, 2, 2, 3, 3, 2, 2]
  :N   => 9241
  :val => [-Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf, -Inf  …  -Inf,…
  :ref => 4231
  :g   => {9241, 14207} undirected simple Int64 graph
  :sd  => [3.784, 0.0, 1.51, 1.7141, 0.4595, 0.0, 0.3103, 0.0, 0.143, 1.34  …  …
  :y   => sparse([1, 2316, 7639, 7835, 2, 2447, 4898, 3, 8189, 8676  …  9237, 1…
  :del => sparse([2316, 7639, 7835, 2447, 4898, 8189, 8676, 1813, 5241, 9018  ……
  :Ng  => 19927
  :sl  => [[0.0, -100.0], [0.0, -100.0, -1.0853], [0.0, -100.0], [0.0, -100.0],…

* Powergrid

In [24]:
using Plasmo
using Ipopt

gamma = 1e5
oms = [1, 4, 7]
max_iter = 1000
scl = 1

g = args[:g]
N_buses = LightGraphs.nv(g)
N_lines = LightGraphs.ne(g)
y = args[:y]
del = args[:del]

powergrid = Plasmo.OptiGraph()
Plasmo.@optinode(powergrid, buses[1:N_buses]) # create node buses
Plasmo.@optinode(powergrid, lines[1:N_lines]) # create transmission lines

node_map_in = Dict((bus,Plasmo.OptiNode[]) for bus in buses)
node_map_out = Dict((bus,Plasmo.OptiNode[]) for bus in buses)

line_map = Dict()
edge_map = Dict()
B = Dict()
angle_rate = Dict()
ngens = Dict()
load_map = Dict()



Dict{OptiNode, Vector{OptiNode}} with 9241 entries:
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) and 0 Constraint(s) => []
  OptiNode w/ 0 Variable(s) an